In [1]:
import pandas as pd
import json
from environs import Env
from graph_functions import query_to_df,start_graph_session, start_graph_session_crashdown
from IPython.display import Image


env = Env()
session = start_graph_session(env) 
#session = start_graph_session_crashdown(env) 

37687


## Get GWAS with available full summary data

In [2]:
query="""
MATCH (gwas:Gwas) -[gwas_to_variant:GWAS_TO_VARIANT]->(variant:Variant)
where toLower(gwas.trait) contains "breast"
and not(toLower(gwas.trait) contains "baby") and not(toLower(gwas.trait) contains "mother")
return distinct gwas.trait, gwas.id, gwas_to_variant.samplesize as SampleSize,
toInteger(gwas.ncase) as N_case, toInteger(gwas.ncontrol) as N_control, gwas.author, gwas.year,
count(gwas_to_variant.pval) as signifSNPs
order by gwas_to_variant.samplesize desc
"""
df = query_to_df(session, query)
all_bc_gwas = list(df['gwas.id'])
df



MATCH (gwas:Gwas) -[gwas_to_variant:GWAS_TO_VARIANT]->(variant:Variant)
where toLower(gwas.trait) contains "breast"
and not(toLower(gwas.trait) contains "baby") and not(toLower(gwas.trait) contains "mother")
return distinct gwas.trait, gwas.id, gwas_to_variant.samplesize as SampleSize,
toInteger(gwas.ncase) as N_case, toInteger(gwas.ncontrol) as N_control, gwas.author, gwas.year,
count(gwas_to_variant.pval) as signifSNPs
order by gwas_to_variant.samplesize desc



,gwas.trait,gwas.id,SampleSize,N_case,N_control,gwas.author,gwas.year,signifSNPs
0,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,337199,5510,331689,Neale,2017.0,7
1,Cancer code self-reported: breast cancer,ukb-a-55,337159,7480,329679,Neale,2017.0,17
2,Breast cancer (Combined Oncoarray; iCOGS; GWAS...,ieu-a-1126,228951,122977,105974,Michailidou K,2017.0,141
3,ER+ Breast cancer (Combined Oncoarray; iCOGS; ...,ieu-a-1127,175475,69501,105974,Michailidou K,2017.0,107
4,ER- Breast cancer (Combined Oncoarray; iCOGS; ...,ieu-a-1128,127442,21468,105974,Michailidou K,2017.0,37
5,Breast cancer (Oncoarray),ieu-a-1129,106776,61282,45494,Michailidou K,2017.0,60
6,Breast cancer (iCOGS),ieu-a-1130,89677,46785,42892,Michailidou K,2017.0,47
7,ER+ Breast cancer (Oncoarray),ieu-a-1132,83691,38197,45494,Michailidou K,2017.0,58
8,ER+ Breast cancer (iCOGS),ieu-a-1133,69970,27078,42892,Michailidou K,2017.0,41
9,ER- Breast cancer (Oncoarray),ieu-a-1135,55149,9655,45494,Michailidou K,2017.0,13


## MR data

In [15]:
# MR
query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id = "ieu-a-1126"  
AND (not (toLower(exposure.trait) contains 'breast')) 
AND mr.pval < 1e-08
RETURN exposure {.id, .trait}, outcome.id, mr {.b, .pval, .method, .selection, .moescore} ORDER BY mr.pval
"""
df = query_to_df(session, query)
df


MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id = "ieu-a-1126"  
AND (not (toLower(exposure.trait) contains 'breast')) 
AND mr.pval < 1e-08
RETURN exposure {.id, .trait}, outcome.id, mr {.b, .pval, .method, .selection, .moescore} ORDER BY mr.pval



,outcome.id,exposure.trait,exposure.id,mr.b,mr.method,mr.selection,mr.pval,mr.moescore
0,ieu-a-1126,Non-cancer illness code self-reported: osteoa...,ukb-a-106,1.778924,FE IVW,DF,1.041688e-298,1
1,ieu-a-1126,Mineral and other dietary supplements: Glucosa...,ukb-a-494,-0.133423,FE IVW,DF,5.154322e-35,1
2,ieu-a-1126,Diagnoses - main ICD10: R14 Flatulence and rel...,ukb-a-584,12.411659,FE IVW,DF,6.556335e-31,1
3,ieu-a-1126,Average weekly champagne plus white wine intake,ukb-a-27,-0.591364,FE IVW,DF,3.926666e-30,1
4,ieu-a-1126,Non-cancer illness code self-reported: pernic...,ukb-a-90,5.220976,FE IVW,DF,3.364830e-26,1
5,ieu-a-1126,Reason for glasses/contact lenses: For just re...,ukb-a-421,2.099325,FE IVW,DF,9.570221e-16,1
6,ieu-a-1126,Waist circumference,ieu-a-102,-0.642405,FE IVW,DF,1.990503e-14,1
7,ieu-a-1126,Diagnoses - main ICD10: I84 Haemorrhoids,ukb-a-539,6.393298,FE IVW,DF,1.474239e-13,1
8,ieu-a-1126,Body mass index,ieu-a-974,-0.788037,FE Egger,HF,9.788118e-13,0.76
9,ieu-a-1126,Non-cancer illness code self-reported: type 2...,ukb-a-75,14.578263,FE IVW,DF,1.636759e-12,1


In [4]:
# MR
# get traits that have pval < 1e-05

query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in {gwas_list} 
AND  not exposure.id  in {gwas_list}
//AND (not (toLower(exposure.trait) contains 'breast')) 
AND mr.pval < 1e-05
with mr, exposure, outcome
ORDER BY mr.pval 
RETURN exposure.id, exposure.trait, exposure.sample_size,
        collect(outcome.id) as outcome_ids, 
        collect(mr.pval) as MR_pvals, collect(mr.b) as MR_beta
        
""".format(gwas_list = all_bc_gwas)
df = query_to_df(session, query, False)

df

,exposure.id,exposure.trait,exposure.sample_size,outcome_ids,MR_pvals,MR_beta
0,ukb-a-479,Types of transport used (excluding work): Walk,335650.0,"[ukb-a-519, ieu-a-1133, ieu-a-1130, ieu-a-1127]","[0.0, 0.0, 1.07475737570078e-94, 1.07287985891...","[-0.057670440232475, -3.04315827052595, -2.226..."
1,ukb-a-564,Diagnoses - main ICD10: M20 Acquired deformiti...,337199.0,"[ukb-a-519, ieu-a-1128]","[0.0, 2.21581458567691e-47]","[0.108120720828772, 13.955307103413]"
2,ukb-a-141,Treatment/medication code: prednisolone,337159.0,"[ieu-a-1127, ieu-a-1129, ieu-a-1131, ieu-a-1126]","[0.0, 3.63681897771967e-08, 1.94394021194942e-...","[-12.9082685230119, -4.68661917520307, -9.9491..."
3,ukb-a-493,Mineral and other dietary supplements: None of...,336314.0,"[ieu-a-1136, ieu-a-1131, ieu-a-1137]","[0.0, 0.0, 4.19019391015668e-210]","[-2.5986128974828, 1.38980136065847, 1.7405472..."
4,ieu-a-1073,Copper,2603.0,[ieu-a-1131],[0.0],[-0.0866186818986572]
...,...,...,...,...,...,...
128,ukb-a-210,Illnesses of mother: Alzheimer's disease/dementia,308780.0,[ieu-a-1126],[6.57162785000218e-06],[-0.566625170720048]
129,ukb-a-436,Vascular/heart problems diagnosed by doctor: A...,336683.0,[ieu-a-1129],[6.77471595152415e-06],[-5.96361006896053]
130,ukb-a-273,Impedance of arm (left),331292.0,[ieu-a-1127],[7.02989960407756e-06],[0.151954240584118]
131,ieu-a-295,Inflammatory bowel disease,2413.0,[ieu-a-1135],[7.81207401376598e-06],[-0.0220616670472059]


In [5]:
# now those traits that appered at least once in something at <1e05, 
# get MR results for those traits with all BC datasets


query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in {gwas_list} 
AND  exposure.id  in {exposure_list}
with mr, exposure, outcome
ORDER BY exposure.trait
RETURN exposure.id, exposure.trait, exposure.sample_size, exposure.sex, 
toInteger(exposure.year) as year, exposure.author as author, exposure.consortium as consortium,
        outcome.id, outcome.sample_size, toInteger(outcome.ncase) as N_case, mr.pval, mr.b, mr.se
""".format(gwas_list = all_bc_gwas,
           exposure_list = list(set(df['exposure.id'])))
df = query_to_df(session, query, True)


#df.to_csv("query_results/bc_all_mr.tsv", sep="\t", index=False)
df


MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in ['ukb-a-519', 'ukb-a-55', 'ieu-a-1126', 'ieu-a-1127', 'ieu-a-1128', 'ieu-a-1129', 'ieu-a-1130', 'ieu-a-1132', 'ieu-a-1133', 'ieu-a-1135', 'ieu-a-1136', 'ieu-a-1131', 'ieu-a-1137', 'ieu-a-1134'] 
AND  exposure.id  in ['ukb-a-543', 'ukb-a-103', 'ieu-a-1051', 'ukb-a-494', 'ukb-a-197', 'ukb-a-178', 'ukb-a-537', 'ukb-a-30', 'ukb-a-260', 'ieu-a-92', 'ukb-a-204', 'ieu-a-1017', 'ukb-a-14', 'ieu-a-101', 'ieu-a-89', 'ukb-a-495', 'ukb-a-354', 'ukb-a-344', 'ukb-a-532', 'ukb-a-531', 'ukb-a-288', 'ukb-a-213', 'ieu-a-785', 'ieu-a-974', 'ukb-a-585', 'ukb-a-416', 'ukb-a-472', 'ukb-a-7', 'ieu-a-68', 'ukb-a-184', 'ieu-a-103', 'ukb-a-11', 'ukb-a-214', 'ukb-a-552', 'ieu-a-792', 'ieu-a-102', 'ukb-a-186', 'ukb-a-555', 'ukb-a-512', 'ukb-a-173', 'ukb-a-487', 'ukb-a-584', 'ukb-a-180', 'ukb-a-323', 'ukb-a-421', 'ukb-a-27', 'ukb-a-91', 'ukb-a-109', 'ukb-a-578', 'ukb-a-420', 'ukb-a-492', 'ieu-a-965', 'ukb-a-479', 'ukb-a-78', 'ukb-a-520', 'uk

,exposure.id,exposure.trait,exposure.sample_size,exposure.sex,year,author,consortium,outcome.id,outcome.sample_size,N_case,mr.pval,mr.b,mr.se
0,ukb-a-320,Age at last live birth,123676.0,Males and Females,2017,Neale,Neale Lab,ukb-a-519,337199.0,5510,2.171147e-01,-0.004151,0.003363
1,ukb-a-320,Age at last live birth,123676.0,Males and Females,2017,Neale,Neale Lab,ukb-a-55,337159.0,7480,5.719176e-06,-0.013066,0.002880
2,ukb-a-320,Age at last live birth,123676.0,Males and Females,2017,Neale,Neale Lab,ieu-a-1127,175475.0,69501,1.401091e-02,-0.390523,0.158944
3,ukb-a-320,Age at last live birth,123676.0,Males and Females,2017,Neale,Neale Lab,ieu-a-1132,83691.0,38197,1.002960e-02,-0.550637,0.213856
4,ukb-a-320,Age at last live birth,123676.0,Males and Females,2017,Neale,Neale Lab,ieu-a-1133,69970.0,27078,3.159866e-01,-0.248122,0.247444
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837,ieu-a-68,Waist circumference,134038.0,Females,2015,Shungin D,None,ukb-a-55,337159.0,7480,7.833466e-01,0.000920,0.003345
1838,ieu-a-102,Waist circumference,60586.0,Males,2013,Randall JC,None,ukb-a-55,337159.0,7480,1.080044e-07,-0.022054,0.004151
1839,ieu-a-68,Waist circumference,134038.0,Females,2015,Shungin D,None,ieu-a-1131,32498.0,14910,2.952982e-01,0.177187,0.169303
1840,ieu-a-103,Waist circumference,73137.0,Females,2013,Randall JC,None,ieu-a-1131,32498.0,14910,6.832442e-10,-0.699319,0.113343


### Extract MR results for a list of exposure GWAS


In [6]:
# read list of exp gwas (EXTRACTED IN R)
risk_factors = list(pd.read_csv("query_results/gwas_rf_list.txt", sep = '\t')['id'])


query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in {gwas_list_outcome} 
AND  exposure.id  in {gwas_list_exposure}
//AND mr.pval < 1e-05
with mr, exposure, outcome
ORDER BY exposure.trait
RETURN exposure.id, exposure.trait, exposure.sample_size, exposure.sex,
        outcome.id, outcome.sample_size, toInteger(outcome.ncase) as N_case, mr.pval, mr.b, mr.se
        
""".format(gwas_list_outcome = all_bc_gwas,
           gwas_list_exposure = risk_factors )
df = query_to_df(session, query, True)


df.to_csv("query_results/gwas_rf_bc_all_mr.tsv", sep="\t", index=False)
df


MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in ['ukb-a-519', 'ukb-a-55', 'ieu-a-1126', 'ieu-a-1127', 'ieu-a-1128', 'ieu-a-1129', 'ieu-a-1130', 'ieu-a-1132', 'ieu-a-1133', 'ieu-a-1135', 'ieu-a-1136', 'ieu-a-1131', 'ieu-a-1137', 'ieu-a-1134'] 
AND  exposure.id  in ['ukb-b-8764', 'ukb-b-7663', 'ukb-a-485', 'ukb-b-2115', 'ukb-b-13932', 'ukb-b-13184', 'ukb-a-483', 'ukb-a-482', 'ukb-a-487', 'ukb-a-512', 'ukb-b-15869', 'ukb-b-11495', 'ukb-b-8865', 'ukb-a-509', 'ukb-a-486', 'ukb-b-2346', 'ukb-b-7337', 'ukb-a-484', 'ukb-b-13702', 'ieu-a-1095', 'ieu-a-1004', 'ukb-b-4206', 'ukb-a-320', 'ukb-b-18541', 'ukb-a-317', 'ukb-b-1209', 'ukb-b-18105', 'ukb-b-17422', 'ukb-a-315', 'ukb-b-9547', 'ukb-b-12405', 'ukb-a-451', 'ukb-b-8727', 'ukb-b-4564', 'ukb-b-8080', 'ukb-a-322', 'ukb-b-9433', 'ukb-a-324', 'ukb-b-4193', 'ukb-b-3768', 'ukb-a-319', 'ukb-a-316', 'ukb-a-323', 'ukb-b-9509', 'ukb-b-16927', 'ukb-b-9487', 'ieu-a-29', 'ieu-a-27', 'ukb-b-13378', 'ieu-b-40', 'ieu-a-974', 'ukb-b-1

,exposure.id,exposure.trait,exposure.sample_size,exposure.sex,outcome.id,outcome.sample_size,N_case,mr.pval,mr.b,mr.se
0,ukb-a-319,Age at first live birth,123846.0,Males and Females,ukb-a-519,337199.0,5510,0.558758,0.001719,0.002879
1,ukb-a-319,Age at first live birth,123846.0,Males and Females,ukb-a-55,337159.0,7480,0.816717,0.001187,0.005120
2,ukb-a-319,Age at first live birth,123846.0,Males and Females,ieu-a-1126,228951.0,122977,0.013006,-0.262657,0.105756
3,ukb-a-319,Age at first live birth,123846.0,Males and Females,ieu-a-1127,175475.0,69501,0.789003,-0.034011,0.124982
4,ukb-a-319,Age at first live birth,123846.0,Males and Females,ieu-a-1128,127442.0,21468,0.618318,-0.171930,0.337429
...,...,...,...,...,...,...,...,...,...,...
596,ukb-a-266,Whole body fat-free mass,331291.0,Males and Females,ieu-a-1136,50225.0,7333,0.000697,-0.342825,0.100251
597,ukb-a-266,Whole body fat-free mass,331291.0,Males and Females,ieu-a-1131,32498.0,14910,0.042964,-0.177672,0.087480
598,ukb-a-266,Whole body fat-free mass,331291.0,Males and Females,ieu-a-1130,89677.0,46785,0.582227,0.038816,0.070557
599,ukb-a-266,Whole body fat-free mass,331291.0,Males and Females,ieu-a-1132,83691.0,38197,0.862899,0.007952,0.046012


### Correlation 

In [7]:
# find which traits are highly correlated with UKB cancers

query = """
MATCH (trait:Gwas)-[gen_cor:GEN_COR]-(assoc_trait:Gwas)
where assoc_trait.id in ['ukb-a-519']
match (trait:Gwas)-[mr:MR_EVE_MR]->(assoc_trait:Gwas)
//where mr.pval < 1e-03
return distinct trait.trait, trait.id, gen_cor.rg, assoc_trait.trait, assoc_trait.id, mr.b, mr.pval
order by gen_cor.rg desc
"""
df = query_to_df(session, query)
df





MATCH (trait:Gwas)-[gen_cor:GEN_COR]-(assoc_trait:Gwas)
where assoc_trait.id in ['ukb-a-519']
match (trait:Gwas)-[mr:MR_EVE_MR]->(assoc_trait:Gwas)
//where mr.pval < 1e-03
return distinct trait.trait, trait.id, gen_cor.rg, assoc_trait.trait, assoc_trait.id, mr.b, mr.pval
order by gen_cor.rg desc



,trait.trait,trait.id,gen_cor.rg,assoc_trait.trait,assoc_trait.id,mr.b,mr.pval
0,Number of self-reported cancers,ukb-a-21,0.8279,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,0.077594,1.850559e-02
1,Had other major operations,ukb-a-326,0.2660,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,0.026946,5.000000e-01
2,Breastfed as a baby,ukb-a-33,0.2471,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.012162,5.060810e-01
3,Headaches for 3+ months,ukb-a-354,0.1980,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.017729,3.182665e-29
4,Qualifications: NVQ or HND or HNC or equivalent,ukb-a-402,0.1885,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,0.026366,5.000000e-01
...,...,...,...,...,...,...,...
260,Reason for glasses/contact lenses: For 'astigm...,ukb-a-422,-0.1279,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.077385,5.000000e-01
261,Diagnoses - main ICD10: I21 Acute myocardial i...,ukb-a-533,-0.1304,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.057651,5.647018e-01
262,Number of days/week of vigorous physical activ...,ukb-a-511,-0.1306,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.006864,3.821609e-02
263,Mineral and other dietary supplements: None of...,ukb-a-493,-0.1405,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.014608,6.539805e-01
